<a href="https://colab.research.google.com/github/DLSeed/DeepLearning/blob/main/ControlNet_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **From https://github.com/lllyasviel/ControlNet**

In [ ]:
#@title ## 1.1. Clone ControlNet
#@markdown Clone ControlNet from GitHub and check for updates. Use textbox below if you want to checkout other branch or old commit. Leave it empty to stay the HEAD on main.

import os
%store -r

!nvidia-smi

root_dir = "/content"
%store root_dir
repo_dir = str(root_dir)+"/ControlNet"
%store repo_dir
models_dir = str(root_dir)+"/ControlNet/models"
%store models_dir


branch = "" #@param {type: "string"}
repo_url = "https://github.com/lllyasviel/ControlNet"

def clone_repo():
  if os.path.isdir(repo_dir):
    print("The repository folder already exists, will do a !git pull instead\n")
    %cd {repo_dir}
    !git pull origin {branch} if branch else !git pull
  else:
    %cd {root_dir}
    !git clone {repo_url} {repo_dir}

if not os.path.isdir(repo_dir):
  clone_repo()

%cd {root_dir}
os.makedirs(repo_dir, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)

if branch:
  %cd {repo_dir}
  status = os.system(f"git checkout {branch}")
  if status != 0:
    raise Exception("Failed to checkout branch or commit")


In [ ]:
#@title ## 1.2. Installing Dependencies
#@markdown This will install required Python packages
import os
%store -r

install_xformers = True #@param {'type':'boolean'}

%cd {repo_dir}

def install_dependencies():
    if install_xformers:
        !pip install gradio -q #gradio isn't required to be mentioned on requirements for apps on HF space
        !sudo apt-get install aria2
        !pip install timm==0.6.12
        !pip install opencv-contrib-python==4.3.0.36
        !pip install pytorch-lightning==1.9.1
        !pip install einops==0.3.0
        !pip install open_clip_torch==2.0.2
        !pip install omegaconf==2.1.1
        !pip install transformers==4.19.2
        !pip install xformers --no-cache-dir
        !pip install --pre -U triton
        !sudo apt-get install aria2
        !sed -i "s@block.launch(server_name='0.0.0.0')@block.launch(debug=True, share=True)@" /content/ControlNet/gradio_canny2image.py
        !sed -i "s@block.launch(server_name='0.0.0.0')@block.launch(debug=True, share=True)@" /content/ControlNet/gradio_annotator.py
        !sed -i "s@block.launch(server_name='0.0.0.0')@block.launch(debug=True, share=True)@" /content/ControlNet/gradio_depth2image.py
        !sed -i "s@block.launch(server_name='0.0.0.0')@block.launch(debug=True, share=True)@" /content/ControlNet/gradio_fake_scribble2image.py
        !sed -i "s@block.launch(server_name='0.0.0.0')@block.launch(debug=True, share=True)@" /content/ControlNet/gradio_hed2image.py
        !sed -i "s@block.launch(server_name='0.0.0.0')@block.launch(debug=True, share=True)@" /content/ControlNet/gradio_hough2image.py
        !sed -i "s@block.launch(server_name='0.0.0.0')@block.launch(debug=True, share=True)@" /content/ControlNet/gradio_normal2image.py
        !sed -i "s@block.launch(server_name='0.0.0.0')@block.launch(debug=True, share=True)@" /content/ControlNet/gradio_pose2image.py
        !sed -i "s@block.launch(server_name='0.0.0.0')@block.launch(debug=True, share=True)@" /content/ControlNet/gradio_scribble2image.py
        !sed -i "s@block.launch(server_name='0.0.0.0')@block.launch(debug=True, share=True)@" /content/ControlNet/gradio_seg2image.py
        !sed -i "s@block.launch(server_name='0.0.0.0')@block.launch(debug=True, share=True)@" /content/ControlNet/gradio_scribble2image_interactive.py
        !sed -i "s@save_memory = False@save_memory = True@" /content/ControlNet/config.py

install_dependencies()

In [ ]:
#@title ## 2.1. Download ControlNet Model 
import os
%store -r

%cd {root_dir}

installModels = []
installv2Models = []

#@markdown ### Available Model
#@markdown You must download the model before launching the Gradio interface
#@markdown Select one of available model to download:


modelUrl = ["", \
            "https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_canny.pth", \
            "https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_depth.pth", \
            "https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_hed.pth", \
            "https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_mlsd.pth", \
            "https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_normal.pth", \
            "https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_openpose.pth", \
            "https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_scribble.pth", \
            "https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_seg.pth"]
modelList = ["", \
             "control_sd15_canny", \
             "control_sd15_depth", \
             "control_sd15_hed", \
             "control_sd15_mlsd", \
             "control_sd15_normal", \
             "control_sd15_openpose", \
             "control_sd15_scribble", \
             "control_sd15_seg" ]
modelName = "control_sd15_canny" #@param ["control_sd15_canny", "control_sd15_depth", "control_sd15_hed", "control_sd15_mlsd", "control_sd15_normal", "control_sd15_openpose", "control_sd15_scribble", "control_sd15_seg"]


if modelName != "":
  installModels.append((modelName, modelUrl[modelList.index(modelName)]))

def install(checkpoint_name, url):
  ext = "pth" if url.endswith(".pth") else "pt"

  hf_token = '' 
  user_header = f"\"Authorization: Bearer {hf_token}\""
  !aria2c --console-log-level=error --summary-interval=10 --header={user_header} -c -x 16 -k 1M -s 16 -d {repo_dir}/models -o {checkpoint_name}.{ext} "{url}"

def install_checkpoint():
  for model in installModels:
    install(model[0], model[1])

install_checkpoint()

In [ ]:
#@title ## 2.1. Download Additional model
#@markdown You must download the model before launching the Gradio interface

!aria2c --console-log-level=error --summary-interval=10 --header={user_header} -c -x 16 -k 1M -s 16 -d /content/ControlNet/annotator/ckpts -o body_pose_model.pth "https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/body_pose_model.pth"
!aria2c --console-log-level=error --summary-interval=10 --header={user_header} -c -x 16 -k 1M -s 16 -d /content/ControlNet/annotator/ckpts -o dpt_hybrid-midas-501f0c75.pt "https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/dpt_hybrid-midas-501f0c75.pt"
!aria2c --console-log-level=error --summary-interval=10 --header={user_header} -c -x 16 -k 1M -s 16 -d /content/ControlNet/annotator/ckpts -o hand_pose_model.pth "https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/hand_pose_model.pth"
!aria2c --console-log-level=error --summary-interval=10 --header={user_header} -c -x 16 -k 1M -s 16 -d /content/ControlNet/annotator/ckpts -o mlsd_large_512_fp32.pth "https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/mlsd_large_512_fp32.pth"
!aria2c --console-log-level=error --summary-interval=10 --header={user_header} -c -x 16 -k 1M -s 16 -d /content/ControlNet/annotator/ckpts -o mlsd_tiny_512_fp32.ptht "https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/mlsd_tiny_512_fp32.pth"
!aria2c --console-log-level=error --summary-interval=10 --header={user_header} -c -x 16 -k 1M -s 16 -d /content/ControlNet/annotator/ckpts -o network-bsds500.pth "https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/network-bsds500.pth"
!aria2c --console-log-level=error --summary-interval=10 --header={user_header} -c -x 16 -k 1M -s 16 -d /content/ControlNet/annotator/ckpts -o upernet_global_small.pth "https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/upernet_global_small.pth"

#Start **ControlNet** ## 

In [ ]:
%cd {repo_dir}

#@markdown Example : If you downloaded the **control_sd15_canny** model you must select **gradio_canny2image.py**

gradioLink = "gradio_canny2image.py" #@param ["gradio_canny2image.py", "gradio_annotator.py", "gradio_depth2image.py", "gradio_fake_scribble2image.py", "gradio_hed2image.py", "gradio_hough2image.py", "gradio_normal2image.py", "gradio_pose2image.py", "gradio_scribble2image.py"]


if gradioLink != "":
   !python {gradioLink}